# Glitching instructions

Here we are going to try to analyze the possible results from glitching under the point of view of the instructions involved

## Preparation

Start with the parameters and the building of our firmware

In [1]:
from hexdump import hexdump
import struct

In [2]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW303'
sample_size = 5

In [3]:
%%bash -s "$PLATFORM"
cd ../hardware/victims/firmware/simpleserial-experiments
make PLATFORM=$1

rm -f -- simpleserial-experiments-CW303.hex
rm -f -- simpleserial-experiments-CW303.eep
rm -f -- simpleserial-experiments-CW303.cof
rm -f -- simpleserial-experiments-CW303.elf
rm -f -- simpleserial-experiments-CW303.map
rm -f -- simpleserial-experiments-CW303.sym
rm -f -- simpleserial-experiments-CW303.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-experiments.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s
rm -f -- simpleserial-experiments.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d
rm -f -- simpleserial-experiments.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i
.
-------- begin --------
avr-gcc (GCC) 5.4.0
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: simpleserial-experiments.c
avr-gcc -c -mmcu=atxmega128d3 -I. -fpack-struct 

In [4]:
%run "Helper_Scripts/Setup_Generic.ipynb"

Serial baud rate = 38400


In [5]:
fw_path = "../hardware/victims/firmware/simpleserial-experiments/simpleserial-experiments-{}.hex".format(PLATFORM)

In [6]:
cw.program_target(scope, prog, fw_path)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2469 bytes


## Glitching

Now we have our target programmed we can start to attack it with our glitching attempts.



In [7]:
def capture():
    global target, scope
    target.flush()
    reset_target(scope)
    # time.sleep(2)
    data = target.read()

    return data.encode("iso-8859-1")  # in this way I don't get weird unicode artefacts for 0x80

def parse(d, debug=False):
        
    if debug:
        hexdump(d)

    if not d.startswith(b'\x00hello'):  # probably the device reseted itself
        raise ValueError(f'received wrong signature from line {d}')
    try:
        values = struct.unpack('B'*32, d[6:])
        return {'r{}'.format(idx):_ for idx, _ in enumerate(values)}
    except:
        return {}

REGISTERS_DEFAULT = {
    'r0': 0x80,
    'r1': 0x01,
    'r2': 0x02,
    'r3': 0x03,
    'r4': 0x04,
    'r5': 0x05,
    'r6': 0x06,
    'r7': 0x07,
    'r8': 0x08,
    'r9': 0x09,
    'r10': 0x0a,
    'r11': 0x0b,
    'r12': 0x0c,
    'r13': 0x0d,
    'r14': 0x0e,
    'r15': 0x0f,
    'r16': 0x10,
    'r17': 0x11,
    'r18': 0x12,
    'r19': 0x13,
    'r20': 0x14,
    'r21': 0x15,
    'r22': 0x16,
    'r23': 0x17,
    'r24': 0x18,
    'r25': 0x19,
    'r26': 0x1a,
    'r27': 0x1b,
    'r28': 0x1c,
    'r29': 0x1d,
    'r30': 0x1e,
    'r31': 0x1f,
}

print(REGISTERS_DEFAULT)

{'r0': 128, 'r1': 1, 'r2': 2, 'r3': 3, 'r4': 4, 'r5': 5, 'r6': 6, 'r7': 7, 'r8': 8, 'r9': 9, 'r10': 10, 'r11': 11, 'r12': 12, 'r13': 13, 'r14': 14, 'r15': 15, 'r16': 16, 'r17': 17, 'r18': 18, 'r19': 19, 'r20': 20, 'r21': 21, 'r22': 22, 'r23': 23, 'r24': 24, 'r25': 25, 'r26': 26, 'r27': 27, 'r28': 28, 'r29': 29, 'r30': 30, 'r31': 31}


In [8]:
from collections import namedtuple
Range = namedtuple('Range', ['min', 'max', 'step'])

if PLATFORM == "CW303":
    scope.glitch.clk_src = "clkgen"
    scope.glitch.output = "glitch_only"
    scope.glitch.trigger_src = "ext_single"
    
    scope.glitch.repeat = 1

    width_range = Range(-45, 45, 0.4)
    offset_range = Range(-45, 45, 0.4)
    ext_range = Range(5000, 15000, 0.1)

    def glitch_on(scope):
        scope.arm()
        scope.io.glitch_lp = True
        scope.io.glitch_hp = False

    def glitch_off(scope):
        scope.io.glitch_hp = False
        scope.io.glitch_lp = False

    glitch_on(scope)
    scope.glitch.ext_offset = 1000
else:
    raise ValueError('This works only for CW303')
    
print(scope)

cwlite Device
gain = 
    mode = high
    gain = 30
    db   = 24.8359375
adc = 
    state      = False
    basic_mode = rising_edge
    timeout    = 2
    offset     = 0
    presamples = 0
    samples    = 5000
    decimate   = 1
    trig_count = 0
clock = 
    adc_src       = clkgen_x4
    adc_phase     = 0
    adc_freq      = 29538459
    adc_rate      = 29538459.0
    adc_locked    = True
    freq_ctr      = 0
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 26
    clkgen_freq   = 7384615.384615385
    clkgen_locked = True
trigger = 
    triggers = tio4
    module   = basic
io = 
    tio1       = serial_rx
    tio2       = serial_tx
    tio3       = high_z
    tio4       = high_z
    pdid       = high_z
    pdic       = high_z
    nrst       = high_z
    glitch_hp  = False
    glitch_lp  = True
    extclk_src = hs1
    hs2        = clkgen
    target_pwr = True
glitch = 
    clk_src     = clkgen
    width   

In [9]:
import random
# we want to try out randomly the parameters
def get_random_from(var):
    start = getattr(var, 'min')
    end   = getattr(var, 'max')
    return random.uniform(start, end)

def glitch():
    global scope
    global target

    offset = get_random_from(offset_range)
    width = get_random_from(width_range)
    ext_offset = get_random_from(ext_range)

    scope.glitch.offset = offset
    scope.glitch.width = width
    scope.glitch.ext_offset = ext_offset

    glitch_on(scope)
    
    data = capture()
    
    glitch_off(scope)

    try:
        registers = parse(data)
    except ValueError as e:
        registers = {}
        
    return offset, width, ext_offset, data, registers

from enum import Enum, auto

class Result(Enum):
    OK = auto()
    RESET = auto()
    GLITCH = auto()

def check(registers):
    is_glitch = registers != REGISTERS_DEFAULT

    if registers == {}:
        return Result.RESET
    elif is_glitch:
        return Result.GLITCH
    else:
        return Result.OK    

In [ ]:
tries = []

for _ in range(100000):
    values = list(glitch())
    values.insert(0, check(values[-1]))
    
    tries.append(values)
    
    if values[0] == Result.GLITCH:
        print(values)
        
    if len(tries) % 1000 == 0:
        print(len(tries))

## Save the data

Since we generate a lot of data maybe would be wise to save it for later dissection

In [213]:
def pack_line(line):
    state = struct.pack('B', line[0].value)
    offset = struct.pack('d', line[1])
    width = struct.pack('d', line[2])
    ext_offset = struct.pack('d', line[3])
    len_message = struct.pack('B', len(line[4]))
    
    return b''.join([state, offset, width, ext_offset, len_message, line[4]])

def save_tries(results):
    import datetime
    now = datetime.datetime.now().date().isoformat()
    with open(f'glitching-cw303-{now}.bin', 'wb') as f:
        for _ in tries:
            f.write(pack_line(_))

In [214]:
save_tries(tries)

## Analysis

It seems that around 9547 we obtained the thing that resembles the most a glitch

```
[[<Result.RESET: 2>,
  21.01109202992602,
  -43.195183661139545,
  9547.433869506192,
  b'\x00hello\x80\x16\x8b\xeb\x0b\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f',
  {}],
```

what seems strange is that the number of cycles seems off with respect to the number of instructions, but if we take into account the way serial communications work then all makes (hopefully) sense: the target uses a baud rate of 38400 baud, this means the in one second can send 38400 bits or 4800 bytes.

Since the way in which is implemented

```c
void output_ch_0(char data)
{
	while(!USART_IsTXDataRegisterEmpty(&USART));
	USART_PutChar(&USART, data);
	return;
}
```

I suppose that each time it sends one character it must wait the physical time it needs the hardware to bit bang the serial port pins: to send one byte takes

In [65]:
seconds_for_byte = 1/48e2

seconds and if the target run at speed of 8MHz (as indicated by the value in ``scope.clock.clkgen_freq``) we have each cycle takes

In [68]:
seconds_for_cycle = 1/8e6

seconds, so at the end each byte takes

In [69]:
seconds_for_byte/seconds_for_cycle

1666.6666666666667

In [109]:
tries_sorted = sorted([_ for _ in tries if _[0] == Result.RESET],
       key=lambda x:x[3])

In [187]:
import numpy as np

def get_range(results, starts):
    offsets = [_[3] for _ in results if _[-2].startswith(starts)]
    mn, mx = (np.amin(offsets), np.amax(offsets)) if len(offsets) > 0 else (0, 0)
    
    # take all the values in the range
    total = len([_ for _ in results if mn <= _[3] <= mx])
    return (mn, mx, mx - mn, len(offsets), total) 

def summary():
    original = b'\x00hello'
    for _ in range(len(original) + 1):
        print('->', _, original[:_], len([__ for __ in tries_sorted if len(__[-2]) == _]))
        for suffix in [ b'h', b'\x00',]:
            needle = original[:_] + suffix
            print(needle, get_range(tries_sorted, needle))

In [182]:
len([_ for _ in tries_sorted if len(_[-2]) == 5])

159

In [188]:
summary()

-> 0 b'' 0
b'h' (0, 0, 0, 0, 0)
b'\x00' (5000.003413898971, 14999.947196710957, 9999.943782811984, 67827, 67827)
-> 1 b'\x00' 0
b'\x00h' (5000.003413898971, 14999.947196710957, 9999.943782811984, 67827, 67827)
b'\x00\x00' (0, 0, 0, 0, 0)
-> 2 b'\x00h' 0
b'\x00hh' (0, 0, 0, 0, 0)
b'\x00h\x00' (0, 0, 0, 0, 0)
-> 3 b'\x00he' 0
b'\x00heh' (5292.143377797993, 5292.143377797993, 0.0, 1, 1)
b'\x00he\x00' (5011.260712362645, 5596.213130920408, 584.9524185577629, 11, 3820)
-> 4 b'\x00hel' 44
b'\x00helh' (5019.676934549942, 5774.751125301518, 755.0741907515758, 2110, 4923)
b'\x00hel\x00' (5588.044796563836, 7517.35410304293, 1929.3093064790937, 5240, 13251)
-> 5 b'\x00hell' 159
b'\x00hellh' (6995.928067051566, 7695.842085154009, 699.9140181024422, 2218, 4795)
b'\x00hell\x00' (7507.457652319772, 9434.997620341102, 1927.5399680213304, 2532, 13153)
-> 6 b'\x00hello' 136
b'\x00helloh' (8971.148062537115, 9615.985475664886, 644.8374131277706, 2194, 4338)
b'\x00hello\x00' (9429.247844731055, 11357.845

We should try to do some statistics about offsets that when cause a reset also generate the same message from the device

In [203]:
def aggregate_by_message(results):
    '''We check if the message is the same and count for how many rows it happens'''
    count = 0
    last = None
    
    stats = []
    
    for _ in results:
        message = _[-2]
        
        if message != last:
            stats.append((count + 1, last))
            count = 0
            last = message
        else:
            count += 1
            
    return stats

In [205]:
aggregate_by_message(tries_sorted)[::-1]

[(1, b'\x00hello\x80\x01\x02'),
 (8,
  b'\x00hello\x80\x01\x02hello\x80\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f'),
 (1, b'\x00hello\x80\x01\x02'),
 (95,
  b'\x00hello\x80\x01\x02hello\x80\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f'),
 (1, b'\x00hello\x80\x01\x02'),
 (199,
  b'\x00hello\x80\x01\x02hello\x80\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f'),
 (1, b'\x00hello\x80\x01\x02'),
 (13,
  b'\x00hello\x80\x01\x02hello\x80\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f'),
 (1, b'\x00hello\x80\x01\x02'),
 (17,
  b'\x00hello\x80\x01\x02hello\x80\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f'),
 (1, b'\x00hello\x80\x01\x02')

In [216]:
needle = b'\x16\x8b\xeb\x0b'
[_ for _ in tries_sorted if needle in _[-2]]

[[<Result.RESET: 2>,
  21.69327084485647,
  -20.814669965822187,
  5009.119906367237,
  b'\x00he\xec\x16\x8b\xeb\x0b\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f',
  {}],
 [<Result.RESET: 2>,
  15.57889282041483,
  -38.57341908462214,
  5011.260712362645,
  b'\x00he\x00\x16\x8b\xeb\x0b\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f',
  {}],
 [<Result.RESET: 2>,
  21.207852685535002,
  -29.036533651545824,
  5021.235092459066,
  b'\x00he\x00\x16\x8b\xeb\x0b\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f',
  {}],
 [<Result.RESET: 2>,
  14.86402956447801,
  21.640087074048978,
  5069.7093460060205,
  b'\x00he\xec\x16\x8b\xeb\x0b\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f',
  {}],
 [<Result.RESET: 2>,
  4

In [222]:
[_ for _ in tries_sorted if _[-2].startswith(b'\x00hello') and not _[-2].startswith(b'\x00hello\x00') and not _[-2].startswith(b'\x00helloh')]

[[<Result.RESET: 2>,
  15.015765217906214,
  -7.223197372542295,
  9141.93785187131,
  b'\x00hello',
  {}],
 [<Result.RESET: 2>,
  2.448840706373929,
  0.012760338921509629,
  9151.665206646347,
  b'\x00hello',
  {}],
 [<Result.RESET: 2>,
  -42.767614085758304,
  8.820012836525578,
  9209.281235780136,
  b'\x00hello',
  {}],
 [<Result.RESET: 2>,
  23.925550038954597,
  -5.28739149128868,
  9217.609192215918,
  b'\x00hello',
  {}],
 [<Result.RESET: 2>,
  -13.081065264206504,
  -0.12153190909945977,
  9231.026457460554,
  b'\x00hello',
  {}],
 [<Result.RESET: 2>,
  -1.0516996657126754,
  -12.329339392262725,
  9241.266984350015,
  b'\x00hello',
  {}],
 [<Result.RESET: 2>,
  -32.88942661697758,
  0.18083216465352336,
  9264.598015110936,
  b'\x00hello',
  {}],
 [<Result.RESET: 2>,
  -0.9644127615517277,
  -0.07050349222181751,
  9274.877107604982,
  b'\x00hello',
  {}],
 [<Result.RESET: 2>,
  28.666951324672098,
  0.17499283860398407,
  9291.00816452946,
  b'\x00hello',
  {}],
 [<Result.R

Can be of particular interest that doesn't exist a reset with string ``b'\x00hello\x80'``

In [235]:
set(__[-2] for __ in [_ for _ in tries_sorted if len(_[-2]) == 7])

{b'\x00hello\x00'}

In [241]:
{__: set([_[-2] for _ in tries_sorted if len(_[-2]) == __]) for __ in range(4, 12)}

{4: {b'\x00he\x0c', b'\x00he,', b'\x00hel'},
 5: {b'\x00hel\x00', b'\x00hel\x04', b'\x00hel\x0c', b'\x00hel,', b'\x00hell'},
 6: {b'\x00hell\x00',
  b'\x00hell\x01',
  b'\x00hell\x03',
  b'\x00hell\x07',
  b'\x00hell\x0f',
  b'\x00hell/',
  b'\x00hello'},
 7: {b'\x00hello\x00'},
 8: {b'\x00hello\x80\x00', b'\x00hello\x80\x01'},
 9: {b'\x00hello\x80\x01\x00', b'\x00hello\x80\x01\x02'},
 10: set(),
 11: set()}

In [56]:
sorted([_ for _ in tries if _[3] > 9547 and _[3] < 9600 and _[0] != Result.OK and _[2] < 0 and _[2] > -20], key=lambda x:x[3])

[[<Result.RESET: 2>,
  -15.613076091195389,
  -16.464205296460847,
  9547.013899776643,
  b'\x00hellohello\x80\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f',
  {}],
 [<Result.RESET: 2>,
  16.967852322546,
  -18.67655545150392,
  9548.640326286522,
  b'\x00hellohello\x80\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f',
  {}],
 [<Result.RESET: 2>,
  6.169023144866891,
  -6.3578982422753185,
  9552.691495147552,
  b'\x00hello\x00',
  {}],
 [<Result.RESET: 2>,
  -43.7297971851836,
  -15.309473728830376,
  9557.056377203364,
  b'\x00hellohello\x80\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f',
  {}],
 [<Result.RESET: 2>,
  -11.211636437438322,
  -13.672595844872372,
  9559.241143137391,
  b'\x00hellohello\x80\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13


```
[<Result.RESET: 2>,
  -21.872949190294726,
  -0.13679399697429773,
  9595.69849505341,
  b'\x00hello\x00',
  {}],
```

In [21]:
[_ for _ in tries if _[0] == Result.GLITCH]

[]

In [23]:
hex(ord('l'))

'0x6c'

In [20]:
bin(0x0e)

'0b1110'

In [12]:
scope.dis()
target.dis()